Kurulum ve geri yükleme

In [1]:
!pip -q install tensorflow tensorflow-datasets

import tensorflow as tf
import tensorflow_datasets as tfds

IMG_SIZE = 224
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

(ds_train, ds_val, ds_test), ds_info = tfds.load(
    "oxford_flowers102",
    split=["train", "validation", "test"],
    as_supervised=True,   # (image, label)
    with_info=True
)

num_classes = ds_info.features["label"].num_classes
print("num_classes:", num_classes)
print("splits:", {k: v.num_examples for k, v in ds_info.splits.items()})


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

...

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/oxford_flowers102/incomplete.GKMTF6_2.1.1/oxford_flowers102-test.tfrecord*…

Generating validation examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/oxford_flowers102/incomplete.GKMTF6_2.1.1/oxford_flowers102-validation.tfr…

Dataset oxford_flowers102 downloaded and prepared to /root/tensorflow_datasets/oxford_flowers102/2.1.1. Subsequent calls will reuse this data.
num_classes: 102
splits: {Split('train'): 1020, Split('test'): 6149, Split('validation'): 1020}


Preprocess and augmentation

In [2]:
from tensorflow.keras import layers

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

def preprocess(image, label, training=False):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32)

    if training:
        image = data_augmentation(image)

    # VGG16 preprocess: BGR + mean subtraction (Keras utility)
    image = tf.keras.applications.vgg16.preprocess_input(image)
    return image, label

train = ds_train.map(lambda x,y: preprocess(x,y,True), num_parallel_calls=AUTOTUNE).shuffle(2048).batch(BATCH_SIZE).prefetch(AUTOTUNE)
val   = ds_val  .map(lambda x,y: preprocess(x,y,False), num_parallel_calls=AUTOTUNE).batch(BATCH_SIZE).prefetch(AUTOTUNE)
test  = ds_test .map(lambda x,y: preprocess(x,y,False), num_parallel_calls=AUTOTUNE).batch(BATCH_SIZE).prefetch(AUTOTUNE)


VGG16 ile Transfer Learning (Stage-1: feature extractor)

In [3]:
base = tf.keras.applications.VGG16(
    include_top=False,
    weights="imagenet",
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)
base.trainable = False

inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)

model = tf.keras.Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.2, min_lr=1e-6),
]

history1 = model.fit(train, validation_data=val, epochs=20, callbacks=callbacks)


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Epoch 1/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 57s 936ms/step - accuracy: 0.0071 - loss: 13.3413 - val_accuracy: 0.0275 - val_loss: 8.2086 - learning_rate: 0.0010
Epoch 2/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 23s 363ms/step - accuracy: 0.0320 - loss: 9.0062 - val_accuracy: 0.1088 - val_loss: 5.8357 - learning_rate: 0.0010
Epoch 3/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 23s 373ms/step - accuracy: 0.0871 - loss: 6.7983 - val_accuracy: 0.2098 - val_loss: 4.2699 - learning_rate: 0.0010
Epoch 4/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 23s 373ms/step - accuracy: 0.1732 - loss: 4.8658 - val_accuracy: 0.3216 - val_loss: 3.2589 - learning_rate: 0.0010
Epoch 5/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 23s 368ms/step - accuracy: 0.2596 - loss: 3.8872 - val_accuracy: 0.4098 - val_loss: 2.6059 - learning_rate: 0.0010
Epoch 6/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 23s 371ms/step - accuracy: 0.3269 - loss: 3.2686 - val_accuracy: 0.4892 - val_loss: 2.2212 - learning_rate: 0.0010
Epoch 7/20
32/32 ━━━━━━━━━━━━━

Fine-tuning (Stage-2: son conv bloklarını aç)

In [4]:
base.trainable = True

# Örnek: block5’i aç, öncesini kapalı tut
set_trainable = False
for layer in base.layers:
    if layer.name.startswith("block5"):
        set_trainable = True
    layer.trainable = set_trainable

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

history2 = model.fit(train, validation_data=val, epochs=15, callbacks=callbacks)


Epoch 1/15
32/32 ━━━━━━━━━━━━━━━━━━━━ 43s 576ms/step - accuracy: 0.8476 - loss: 0.5357 - val_accuracy: 0.7510 - val_loss: 1.1428 - learning_rate: 1.0000e-05
Epoch 2/15
32/32 ━━━━━━━━━━━━━━━━━━━━ 24s 399ms/step - accuracy: 0.8652 - loss: 0.4678 - val_accuracy: 0.7578 - val_loss: 1.1740 - learning_rate: 1.0000e-05
Epoch 3/15
32/32 ━━━━━━━━━━━━━━━━━━━━ 24s 392ms/step - accuracy: 0.9102 - loss: 0.3447 - val_accuracy: 0.7559 - val_loss: 1.2329 - learning_rate: 1.0000e-05
Epoch 4/15
32/32 ━━━━━━━━━━━━━━━━━━━━ 42s 398ms/step - accuracy: 0.9025 - loss: 0.3163 - val_accuracy: 0.7686 - val_loss: 1.1959 - learning_rate: 2.0000e-06
Epoch 5/15
32/32 ━━━━━━━━━━━━━━━━━━━━ 25s 412ms/step - accuracy: 0.9093 - loss: 0.3399 - val_accuracy: 0.7706 - val_loss: 1.1454 - learning_rate: 2.0000e-06
Epoch 6/15
32/32 ━━━━━━━━━━━━━━━━━━━━ 24s 396ms/step - accuracy: 0.9116 - loss: 0.2532 - val_accuracy: 0.7686 - val_loss: 1.1508 - learning_rate: 1.0000e-06


Test değerlendirme + confusion matrix

In [5]:
test_loss, test_acc = model.evaluate(test)
print("Test accuracy:", test_acc)


193/193 ━━━━━━━━━━━━━━━━━━━━ 39s 202ms/step - accuracy: 0.7159 - loss: 1.4845
Test accuracy: 0.7183281779289246


model save

In [6]:
MODEL_NAME = "vgg16_flowers102.keras"

model.save(MODEL_NAME)


In [7]:
!ls -lh


total 111M
drwxr-xr-x 1 root root 4.0K Dec  9 14:42 sample_data
-rw-r--r-- 1 root root 111M Jan 15 09:34 vgg16_flowers102.keras


bilgisayarına indir

In [8]:
from google.colab import files
files.download("vgg16_flowers102.keras")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>